


# Assignment 5 on Natural Language Processing

## Date : 3rd Nov, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to explore various language models specifically LSTM based and transformer. We will explore how the size of the model effects the sequence generated. We will see both character based and word based models.



Please submit with outputs. Submissions without predicted outputs will be penalized.

# Word Based LSTM model

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras

Do basic pre processing which includes lowering etc
Check the dataset and apply suitable preprocessing.

In [2]:
# Load the data and preprocess data and store corpus in raw_text
import re
import nltk
nltk.download('punkt') # For tokenizers
f = open("corpus.txt", "r",encoding='utf8')
data=f.read()

data=data.lower()
#data=data.replace('_',' ')
#data=data.replace('—',' ')
sentdata=data.split('\n')
#print(sentdata)
raw_text=""

raw_data = []
tokenizer = nltk.RegexpTokenizer(r"\w+")
sentdata=data.split('\n')

for sentence in sentdata:
    
    sentence=sentence.lower()
    new_words = tokenizer.tokenize(sentence)
    fin_words=[]
    for word in new_words:
        getVals = list([val for val in word 
               if val.isalpha()])
        result="".join(getVals)
        fin_words.append(result)
    raw_data.append(fin_words)
    raw_text=""
    for element in raw_data:
      for val in element:
        raw_text=raw_text + " "+val
      raw_text=raw_text+"\n"


print(raw_data)
unique_tokens=[]
for temp in raw_data:
  for word in temp:
    if word not in unique_tokens:
      unique_tokens.append(word)
  
print(raw_text)
print('number of unique tokens :', len(unique_tokens)) 




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[['chapter', 'i'], ['down', 'the', 'rabbit', 'hole', 'alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting', 'by', 'her', 'sister', 'on', 'the', 'bank', 'and', 'of', 'having', 'nothing', 'to', 'do', 'once', 'or', 'twice', 'she', 'had', 'peeped', 'into', 'the', 'book', 'her', 'sister', 'was', 'reading', 'but', 'it', 'had', 'no', 'pictures', 'or', 'conversations', 'in', 'it', 'and', 'what', 'is', 'the', 'use', 'of', 'a', 'book', 'thought', 'alice', 'without', 'pictures', 'or', 'conversations', 'so', 'she', 'was', 'considering', 'in', 'her', 'own', 'mind', 'as', 'well', 'as', 'she', 'could', 'for', 'the', 'hot', 'day', 'made', 'her', 'feel', 'very', 'sleepy', 'and', 'stupid', 'whether', 'the', 'pleasure', 'of', 'making', 'a', 'daisy', 'chain', 'would', 'be', 'worth', 'the', 'trouble', 'of', 'getting', 'up', 'and', 'picking', 'the', 'daisies', 'when', 'suddenly', 'a

In [3]:
# Hyperparameters of the model
vocab_size = len(unique_tokens) # choose based on statistics
oov_tok = '<OOV>'
embedding_dim = 100
padding_type='post'
trunc_type='post'

In [4]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts([raw_text])
word_index = tokenizer.word_index
print(word_index)


{'<OOV>': 1, 'the': 2, 'and': 3, 'to': 4, 'a': 5, 'it': 6, 'she': 7, 'i': 8, 'of': 9, 'said': 10, 'you': 11, 'alice': 12, 'in': 13, 'was': 14, 'that': 15, 'as': 16, 'her': 17, 't': 18, 'at': 19, 's': 20, 'on': 21, 'all': 22, 'with': 23, 'had': 24, 'but': 25, 'for': 26, 'so': 27, 'they': 28, 'be': 29, 'not': 30, 'very': 31, 'what': 32, 'this': 33, 'little': 34, 'he': 35, 'out': 36, 'is': 37, 'one': 38, 'down': 39, 'up': 40, 'there': 41, 'if': 42, 'his': 43, 'then': 44, 'about': 45, 'no': 46, 'them': 47, 'know': 48, 'like': 49, 'were': 50, 'would': 51, 'went': 52, 'again': 53, 'herself': 54, 'do': 55, 'have': 56, 'when': 57, 'or': 58, 'could': 59, 'queen': 60, 'thought': 61, 'off': 62, 'time': 63, 'how': 64, 'me': 65, 'into': 66, 'see': 67, 'well': 68, 'did': 69, 'm': 70, 'who': 71, 'can': 72, 'king': 73, 'your': 74, 'don': 75, 'now': 76, 'turtle': 77, 'by': 78, 'began': 79, 'my': 80, 'its': 81, 'll': 82, 'an': 83, 'way': 84, 'hatter': 85, 'mock': 86, 'quite': 87, 'gryphon': 88, 'are': 8

In [47]:
seq_length = 50
tokens = tokenizer.texts_to_sequences([raw_text])[0]

In [48]:
dataX = []
dataY = []

for i in range(0, len(tokens) - seq_length-1 , 1):
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  if seq_out==1: #Skip samples where target word is OOV
    continue
    
  dataX.append(seq_in)
  dataY.append(seq_out)
 
N = len(dataX)
print ("Total training data size: ", N)

Total training data size:  24988


In [49]:
X = numpy.array(dataX)

# one hot encode the output variable
y = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [50]:
# with embedding
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(vocab_size, activation='softmax')
])
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 100)           256900    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               84480     
_________________________________________________________________
dense_3 (Dense)              (None, 2569)              331401    
Total params: 672,781
Trainable params: 672,781
Non-trainable params: 0
_________________________________________________________________


In [51]:
# Use validation split of 0.2 while training
model.fit(X, y, epochs=120, batch_size=128,validation_split=0.2) 

Epoch 1/120
157/157 [==============================] - 3s 20ms/step - loss: 6.3878 - accuracy: 0.0511 - val_loss: 6.2114 - val_accuracy: 0.0884
Epoch 2/120
157/157 [==============================] - 3s 16ms/step - loss: 5.9076 - accuracy: 0.0588 - val_loss: 6.1374 - val_accuracy: 0.0886
Epoch 3/120
157/157 [==============================] - 3s 16ms/step - loss: 5.7909 - accuracy: 0.0620 - val_loss: 6.1137 - val_accuracy: 0.0924
Epoch 4/120
157/157 [==============================] - 3s 16ms/step - loss: 5.6733 - accuracy: 0.0711 - val_loss: 6.0664 - val_accuracy: 0.1102
Epoch 5/120
157/157 [==============================] - 3s 16ms/step - loss: 5.5364 - accuracy: 0.0867 - val_loss: 5.9929 - val_accuracy: 0.1190
Epoch 6/120
157/157 [==============================] - 3s 17ms/step - loss: 5.3965 - accuracy: 0.1034 - val_loss: 5.9561 - val_accuracy: 0.1240
Epoch 7/120
157/157 [==============================] - 3s 17ms/step - loss: 5.2692 - accuracy: 0.1136 - val_loss: 5.9318 - val_accuracy:

In [52]:
reverse_word_map = tokenizer.index_word


In [53]:
# Complete the code to return next n words greedily
def next_tokens(input_str, n): 
#	print ("Seed: \n",  input_str)
	tempstr=input_str.split(' ')
	#print(len(tempstr))
	vals=[]
	for word in tempstr:
		if word!='':
			if word in word_index:
				vals.append(word_index[word])
			else:
				vals.append(word_index['<OOV>'])
	
	
	
	final_string=""
	for i in range(n):
		arr=np.reshape(vals,(1,len(vals),1))
		prediction = model.predict(arr,verbose=0)
		ind=np.argmax(prediction)
		word=reverse_word_map[ind]
		# get next word index. Use reverse_word_map to get the word
		final_string=final_string+" "+word
		vals.append(ind)
		vals=vals[1:len(vals)]
	return final_string.strip()

In [15]:
# pick a random seed
import random
random.seed()
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
#print(pattern)
input_str = ' '.join([reverse_word_map[value] for value in pattern])
print(input_str)
print('predicted output :')
print(next_tokens( input_str , 10))

don t give birthday presents like that but she did not venture to say it out loud thinking again the duchess asked with another dig of her sharp little chin i ve a right to think said alice sharply for she was beginning to feel a little worried just about
predicted output :
as much right said the duchess as pigs have to


In [54]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

input_str=input_str.lower()
tokens=tokenizer.texts_to_sequences([input_str])[0]
temp=tokens[0:50]
#print(tokens)
'''
tokenizer = nltk.RegexpTokenizer(r"\w+")
new_words = tokenizer.tokenize(input_str)

fin_words=[]
temp_data=[]
for word in new_words:
    getVals = list([val for val in word 
                    if val.isalpha()])
    result="".join(getVals)
    fin_words.append(result)
    temp_data.append(fin_words)
'''
text=""
i=0
for word in temp:
  text=text + " "+reverse_word_map[word]

text.strip()
print(text)
print('predicted output :')
# Use first 50 tokens from given input_str as input.(Use tokenizer to split to take first 50)
print(next_tokens( text , 10))

 the boy laughed at the fright he had caused this time the <OOV> left angrily the <OOV> day as the boy went up the small <OOV> he suddenly saw a <OOV> <OOV> his sheep he cried as hard as he could <OOV> <OOV> <OOV> <OOV> but not a <OOV> <OOV>
predicted output :
you like you want to go back to the same


# Character based LSTM Model 1

In [18]:
# User the preprocess data and create raw_text
data=data.lower()
data=data.replace('_',' ')
data=data.replace('—',' ')
raw_text=data
# create mapping of unique characters to integers
chars = sorted(list(set(raw_text)))

char_to_int = dict((cha, integer) for integer, cha in enumerate(chars))

In [19]:
# Print the total characters and character vacob size
n_chars = len(raw_text)
n_vocab = len(chars)
print('Total characters :',n_chars)
print('Total Vocab :',n_vocab)

Total characters : 142477
Total Vocab : 44


In [20]:
'''
Prepare dataset where the input is sequence of 100 characters and target is next character.
'''
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_out = raw_text[i + seq_length]
    seq_in = raw_text[i:i + seq_length]
    dataY.append(char_to_int[seq_out])
    dataX.append([char_to_int[char] for char in seq_in])

n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  142377


In [21]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX,(n_patterns,seq_length,1))

# one hot encode the output variable
dataY = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [22]:
embedding_dim =100
max_length =100

In [23]:
model = Sequential()
model.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          4400      
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 44)                11308     
Total params: 381,276
Trainable params: 381,276
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(X, y, epochs=20, batch_size=128)

Epoch 1/20
1113/1113 [==============================] - 20s 18ms/step - loss: 2.2062
Epoch 2/20
1113/1113 [==============================] - 21s 19ms/step - loss: 1.7013
Epoch 3/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.5062
Epoch 4/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.3858
Epoch 5/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.3058
Epoch 6/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.2443
Epoch 7/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.1911
Epoch 8/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.1474
Epoch 9/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.1120
Epoch 10/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.0753
Epoch 11/20
1113/1113 [==============================] - 20s 18ms/step - loss: 1.0423
Epoch 12/20
1113/1113 [==============================] - 20s 18

In [26]:
#implement mapping of integer to character
int_to_char =int_to_char = dict((integer, cha) for integer, cha in enumerate(chars))

In [38]:
'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars1(pattern,x):
	final_string=""
	for i in range(x):
		# Complete Code
		x = numpy.reshape(pattern, (1, len(pattern), 1))
		prediction = model.predict(x, verbose=0)
		ind = np.argmax(prediction)
		ch=int_to_char[ind]
		final_string=final_string+ch
		pattern.append(ind)
		pattern=pattern[1:len(pattern)]
	

	return final_string

In [39]:
# pick a random seed
random.seed()
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])
print(input_str)
print('predicted output: ')
print(predict_next_100_chars1(pattern,200))

hing the game.” the queen smiled and passed on.
“who  are  you talking to?” said the king, going up 
predicted output: 
and said, “i should think you can  have no mice, and the mouse doesn’t like cats is to be a little bit,” said the mock turtle.
“what  are  you  mistape then i should think you can  i have to bean the 


In [44]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."
input_str=input_str.lower()
temp=input_str[0:100]
pattern=[]
for ch in temp:
  pattern.append(char_to_int[ch])

 # Use first 100 characeters from given input_str as input and generate next 200 characters.

print(temp)
print('predicted output:')
print(predict_next_100_chars1(pattern,200))

the boy laughed at the fright he had caused. this time, the villagers left angrily. the third day, a
predicted output:
nd not the time there was no second to say that it was the white rabbit, and the poor little thing was so much sudden a child like the words “and that is the same thing as ‘i was a child!” said the mo


## Character based LSTM Model 2


In [30]:
model1 = Sequential()
model1.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model1.add(LSTM(256, input_shape=(X.shape[1], embedding_dim),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')

In [31]:
model1.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
2225/2225 [==============================] - 51s 23ms/step - loss: 2.0698
Epoch 2/20
2225/2225 [==============================] - 50s 23ms/step - loss: 1.5526
Epoch 3/20
2225/2225 [==============================] - 51s 23ms/step - loss: 1.3854
Epoch 4/20
2225/2225 [==============================] - 51s 23ms/step - loss: 1.2907
Epoch 5/20
2225/2225 [==============================] - 50s 23ms/step - loss: 1.2226
Epoch 6/20
2225/2225 [==============================] - 50s 23ms/step - loss: 1.1679
Epoch 7/20
2225/2225 [==============================] - 50s 23ms/step - loss: 1.1248
Epoch 8/20
2225/2225 [==============================] - 50s 23ms/step - loss: 1.0872
Epoch 9/20
2225/2225 [==============================] - 50s 23ms/step - loss: 1.0535
Epoch 10/20
2225/2225 [==============================] - 51s 23ms/step - loss: 1.0269
Epoch 11/20
2225/2225 [==============================] - 50s 23ms/step - loss: 1.0036
Epoch 12/20
2225/2225 [==============================] - 50s 23

In [41]:
# Generate the sequence similar to above methods

'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars2(pattern,x):
	final_string=""
	for i in range(x):
		# Complete Code
		x = numpy.reshape(pattern, (1, len(pattern), 1))
		prediction = model1.predict(x, verbose=0)
		ind = np.argmax(prediction)
		ch=int_to_char[ind]
		final_string=final_string+ch
		pattern.append(ind)
		pattern=pattern[1:len(pattern)]
	return final_string

In [42]:
# pick a random seed
random.seed()
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])
print(input_str)
print('predicted output:')
print(predict_next_100_chars2(pattern,200))

he rattling teacups would change to tinkling sheep-bells, and the queen’s shrill cries to the voice 
predicted output:
of the sea, and the top of the way the white rabbit with the little golden key, and the three gardeners who were little shriek fast as she could go, and the three gardeners when she was not a minute o


In [45]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

input_str=input_str.lower()
temp=input_str[0:100]
pattern=[]
for ch in temp:
  pattern.append(char_to_int[ch])

 # Use first 100 characeters from given input_str as input and generate next 200 characters.

print(temp)
print('predicted output:')
print(predict_next_100_chars2(pattern,200))

the boy laughed at the fright he had caused. this time, the villagers left angrily. the third day, a
predicted output:
nd she tried to see that she was not a minute or two, with a sigh: “it’s a very  prower tone, and the moment it was to see if it was the first tarts of the sea, and the top of the way the white rabbit


# Questions

**Question:** What are your observations based on the model(all) outputs on train data(in domain) vs unseen data(out of domain) ?

**Answer:**
The models are able to give more coherent output on the train data(in domain) than the the unseen data(out of domain). In the predictions of the unseen data some words are introduced which have no relation with the unseen data. This is understood because the out of domain input contains many out of vocabulary tokens which affect model performance in predicting the next words.

**Question:** What did you observe in the outputs of char LSTM model1 vs char LSTM model2 ?




**Answer:**
The LSTM model2 produces better output and maintains more consistency in linking the input sentence and predicted output.
For example for LSTM1 : **said the king, going up** and said, “i should think you can (where the bold text is the input sentence and following that is predicted text) has no proper connection. But for LSTM2: **queen’s shrill cries to the voice**  of the sea, and the top of the way (where the bold text is the input sentence and following that is predicted text) is more connected. For the out of domain example both models do not perform well. In both models outputs 'a'is followed by 'nd' in the prediction where it should have been followed by a 's'. LSTM model2 has more number of layers as compared to LSTM model1 and this helps in better learning the dataset and a better prediction output.

# Bonus (Not to be graded)

## Transformer based language model (Bert)
You can run the below cells to predict the next word using pretrained BERT model. 

In [ ]:
!pip install transformers

In [ ]:
import os
import torch
import string
from transformers import BertTokenizer, BertForMaskedLM

In [ ]:
def load_model(model_name):
  try:
    if model_name.lower() == "bert":
      bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()
      return bert_tokenizer,bert_model
  except Exception as e:
    pass

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
  ignore_tokens = string.punctuation + '[PAD]'
  tokens = []
  for w in pred_idx:
    token = ''.join(tokenizer.decode(w).split())
    if token not in ignore_tokens:
      tokens.append(token.replace('##', ''))
  return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
  text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
  # if <mask> is the last token, append a "." so that models dont predict punctuation.
  if tokenizer.mask_token == text_sentence.split()[-1]:
    text_sentence += ' .'
  input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
  mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
  return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
  input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
  with torch.no_grad():
    predict = bert_model(input_ids)[0]
    print(predict.shape)
    
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
  return {'bert': bert}

In [ ]:
def get_prediction_eos(input_text):
  try:
    input_text += ' <mask>'
    res = get_all_predictions(input_text, top_clean=int(top_k))
    return res
  except Exception as error:
    pass

In [ ]:
# Below code predicts the next top_k words. You can modify this to get next n words using top_k=1 and greedy decoding it. 
top_k= 3
print('Predict next ', top_k, ' words')
model_name = 'BERT'
bert_tokenizer, bert_model  = load_model(model_name) 
input_text = "" ### GIVE YOUR INPUT STRING HERE
res = get_prediction_eos(input_text)
answer = []
print(res['bert'].split("\n"))
for i in res['bert'].split("\n"):
  answer.append(i)
  answer_as_string = "    ".join(answer)

print(answer_as_string)